In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from TextureClassifier import TextureClassifier
import os
import logging
import numpy as np
import socket
hostname = socket.gethostname()

import mxnet as mx

In [5]:
logging.basicConfig(level=logging.DEBUG)

# stack = 'MD589'
# sec = 165

stack = 'MD594'
sec = 289

if hostname.startswith('gcn'):
    data_dir = '/oasis/projects/nsf/csd395/yuncong/jiaxu_sample_patches'
    model_dir = '/oasis/projects/nsf/csd395/yuncong/jiaxu_repo/model_publish'    
elif hostname == 'yuncong-MacbookPro':
#     data_dir = '/home/yuncong/jiaxu_sample_patches'
    data_dir = '/home/yuncong/CSHL_data_patches/%s_patches/%04d_reduced'%(stack, sec)
    model_dir = '/home/yuncong/jiaxu_repo/model_publish' 
    
# p_labels = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

prefix = 'inception-stage1'
n_iter = 6

In [11]:
img_filenames = sorted(os.listdir(data_dir))
with open('/home/yuncong/CSHL_data_patches/%s_%04d_test.lst'%(stack, sec), 'w') as lst_file:
    for index, img_filename in enumerate(img_filenames):
        lst_file.write('%s\t0\t%s\n'%(index, img_filename))

In [12]:
os.system('im2rec /home/yuncong/CSHL_data_patches/%(stack)s_%(sec)04d_test.lst \
/home/yuncong/CSHL_data_patches/%(stack)s_patches/%(sec)04d_reduced/ \
/home/yuncong/CSHL_data_patches/%(stack)s_%(sec)04d_test.rec' % \
          {'stack': stack,
           'sec': sec})
# scanning 1000 images take 1.8s

0

In [8]:
batch_size = 32
init_model = mx.model.FeedForward.load(os.path.join(model_dir, prefix), n_iter, ctx=mx.gpu())
test_iter = mx.io.ImageRecordIter(
    path_imgrec='/home/yuncong/CSHL_data_patches/%(stack)s_%(sec)04d_test.rec' % {'stack': stack, 'sec':sec},
    batch_size=batch_size,
    data_shape=(3, 224, 224),
    mean_img=os.path.join(model_dir, 'mean_224.nd'),
)
probs = init_model.predict(test_iter)

In [9]:
# prediction_map = zip(np.argmax(probs, axis=1), map(lambda x: int(x[:-4]), img_filenames))
np.save('/home/yuncong/CSHL_data_patches/%(stack)s_%(sec)04d_test_prediction_map.npy'% {'stack': stack, 'sec':sec}, 
        probs)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
test_iter.reset()
lables = np.concatenate([b.label[0].asnumpy() for b in test_iter]).astype(np.int)

In [ ]:
np.count_nonzero(np.argmax(preds, axis=1) != lables) / float(preds.shape[0])

In [ ]:
# tc.mx_confusion()